# 111 - Content Based Filtering - manual feature engineering

MovieLens 데이터셋을 사용하여 사용자에게 새로운 영화를 추천하는 Content-Based Filtering 추천 시스템입니다. 영화의 장르를 기반으로 사용자 프로필을 생성하고, 사용자가 아직 평가하지 않은 영화 중에서 개인화된 추천을 수행합니다.

## Content-Based Recommender System algorithm

Step 1: 사용자의 영화 평가를 근거로 가중 장르 행렬 작성 - 사용자가 선호하는 장르의 가중치 계산  
    - 각 사용자가 본 영화에 대한 평점을 영화의 장르별로 합산합니다.   
    - 합산된 평점을 각 장르별로 정규화하여 가중치를 계산합니다.  
    
Step 2: 가중 장르 행렬로부터 사용자 프로파일 작성 - 각 사용자의 장르 선호도를 나타내는 사용자 프로파일 작성  
    - 각 사용자의 가중 장르 행렬을 기반으로 사용자 프로파일을 생성합니다.  
    - 사용자 프로파일은 사용자가 선호하는 각 장르의 가중치를 포함합니다.  
    
Step 3: 사용자가 보지 않은 영화에 사용자 프로파일 적용하여 추천 영화 리스트 작성
    -사용자가 보지 않은 영화의 장르 행렬을 작성합니다.  
    -사용자 프로파일과 영화의 장르 행렬을 기반으로 각 영화에 대한 예측 평점을 계산합니다.  
    
예측 평점이 높은 영화를 사용자에게 추천합니다.     
사용자의 영화 평가 데이터를 기반으로 하여 사용자가 선호하는 장르를 파악하고, 이를 바탕으로 새로운 영화를 추천하는 방식입니다.

In [2]:
# 필요한 Library import
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import warnings

# 경고 메시지를 무시하도록 설정
warnings.filterwarnings('ignore')

- `ratings.csv` 데이터셋은 사용자들이 다양한 영화에 대해 매긴 평점 정보를 포함
    - userId: 사용자의 고유 ID
    - movieId: 영화의 고유 ID
    - rating: 사용자가 영화에 매긴 평점
    - timestamp: 사용자가 평점을 매긴 시간 정보. Unix 타임스탬프 형식.
 <br><br>
- `movies.csv` 데이터셋은 각 영화의 고유 정보와 장르를 포함  
    - movieId: 영화의 고유 ID  
    - title: 영화의 제목  
    - genres: 영화의 장르. 여러 장르가 쉼표로 구분되어 있습니다.

In [3]:
# 사용자 영화 평점 데이터 로드
ratings = pd.read_csv('data/ml-latest-small/ratings.csv', index_col=0)
print(ratings.shape)
ratings.head(3)

(100836, 3)


,movieId,rating,timestamp
userId,,,
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224


In [4]:
# 영화 특성 데이터 로드
df_movies = pd.read_csv('data/ml-latest-small/movies.csv', index_col=0)
print(df_movies.shape)
df_movies.head(3)

(9742, 2)


,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance


In [9]:
# ratings에서 중복 movieId를 제거합니다.
unique_movie_ids_in_ratings = ratings['movieId'].unique()

# movies dataframe에서 사용자 rating이 있는 movie만 골라냅니다.
df_rated_movies = df_movies[df_movies.index.isin(unique_movie_ids_in_ratings)]

print(df_rated_movies.shape)
df_rated_movies.head(3)

(9724, 2)


,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance


In [10]:
# 평가된 영화 데이터프레임에서 영화 제목을 가져옴
movie_names_list = df_rated_movies['title'].values

# 영화 제목 목록의 길이를 출력
print(len(movie_names_list))

# 영화 제목 목록을 출력
print(movie_names_list)

9724
['Toy Story (1995)' 'Jumanji (1995)' 'Grumpier Old Men (1995)' ...
 'Flint (2017)' 'Bungo Stray Dogs: Dead Apple (2018)'
 'Andrew Dice Clay: Dice Rules (1991)']


### 사용자별 영화 평가 Matrix 생성 

사용자가 선호하는 장르의 가중치 계산

In [8]:
# ratings 데이터프레임을 변환하여 movieId를 열로, rating을 값으로 하는 데이터프레임으로 변환
df_users_seen_movies = ratings.pivot(columns='movieId', values='rating')

# 변환된 데이터프레임의 shape을 출력
print(df_users_seen_movies.shape)

# 결측값(NaN)을 0으로 채움
df_users_seen_movies.fillna(0, inplace=True)

(610, 9724)


In [13]:
# 컬럼명을 영화명으로 replace
df_users_seen_movies.columns = movie_names_list
df_users_seen_movies.head(3)

,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# 사용자 영화평 matrix 생성
users_seen_movies = df_users_seen_movies.values
print(users_seen_movies.shape)

(610, 9724)


### 영화 특성 Matrix 작성

In [12]:
# 영화 데이터에서 모든 장르를 추출하여 중복을 제거한 후 정렬된 리스트를 생성합니다.
features = sorted(list(set([genre for sublist in df_movies['genres'].str.split("|") for genre in sublist])))
print(features)

['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [13]:
# features라는 리스트에 있는 각 genre 요소가 문자열 genres에 포함되어 있는지 여부를 확인하는 리스트 컴프리헨션
genres = 'Adventure|Animation|Children|Comedy|Fantasy'
print([genre in genres for genre in features])

[False, False, True, True, True, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False]


In [14]:
# 위의 list comprehesion 결과를 multi-hot encoding으로 표현
np.array([genre in genres for genre in features], dtype=np.float32)

array([0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [15]:
# all_eatures 리스트에 있는 각 genre 요소가 문자열 genres에 포함되어 있는지 여부를 multi-hot encoding 변환하는 함수
def genre_vector(genres, all_features):
    return np.array([genre in genres for genre in all_features], dtype=np.float32)

# 'genre_vec' multi-hot encoding 열을 추가합니다.
df_rated_movies['genre_vec'] = [genre_vector(genres, features) for genres in df_rated_movies['genres']]

# multi-hot encoding된 결과 출력
df_rated_movies.head(3)

,title,genres,genre_vec
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
2,Jumanji (1995),Adventure|Children|Fantasy,"[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Grumpier Old Men (1995),Comedy|Romance,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [16]:
# 각 영화의 장르 벡터를 수직으로 쌓아 새로운 데이터프레임 생성
df_movies_features = pd.DataFrame(np.vstack(df_rated_movies['genre_vec']), columns=features)

print(df_movies_features.shape)
df_movies_features.head()

(9724, 20)


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# 각 영화에 대한 장르 특성 maxtrix 생성
movies_features = df_movies_features.values
print(movies_features.shape)

(9724, 20)


## step1) 사용자 특성 매트릭스 (weighted genre matrix) 계산

사용자의 영화 평가를 근거로 가중 장르 행렬 작성 - 사용자가 선호하는 장르의 가중치 계산

In [18]:
users_seen_movies.shape, movies_features.shape

((610, 9724), (9724, 20))

In [19]:
# 사용자 특성 행렬 생성
# 각 사용자가 본 영화의 평점과 영화의 장르 특성을 행렬곱하여 사용자의 특성을 계산합니다.
# 즉, 사용자가 본 영화의 장르 가중치를 합산하여 사용자의 장르 선호도를 나타내는 벡터를 생성합니다.
# users_seen_movies: 사용자별로 본 영화와 그 평점이 포함된 행렬
# movies_features: 각 영화의 장르 특성이 포함된 행렬
users_features = np.matmul(users_seen_movies, movies_features)

# 계산된 사용자 특성 행렬 
users_features

array([[   0. ,  389. ,  373. , ...,  228. ,   99. ,   30. ],
       [   0. ,   43.5,   12.5, ...,   37. ,    4.5,    3.5],
       [   0. ,   50. ,   30. , ...,   29. ,    2.5,    0. ],
       ...,
       [   0. ,  922.5,  583. , ...,  916. ,   68. ,   29. ],
       [   0. ,   34. ,   32. , ...,   46. ,   14. ,    4. ],
       [   0. , 1861.5,  989.5, ..., 1822.5,  177.5,  123.5]])

In [20]:
# 사용자의 장르 선호도를 나타내는 사용자 특성 행렬을 데이터프레임으로 변환
df_users_features = pd.DataFrame(users_features, columns=features)

df_users_features.head(3)

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,389.0,373.0,136.0,191.0,355.0,196.0,0.0,308.0,202.0,5.0,59.0,0.0,103.0,75.0,112.0,169.0,228.0,99.0,30.0
1,0.0,43.5,12.5,0.0,0.0,28.0,38.0,13.0,66.0,0.0,0.0,3.0,15.0,0.0,8.0,4.5,15.5,37.0,4.5,3.5
2,0.0,50.0,30.0,2.0,2.5,9.0,1.0,0.0,12.0,13.5,0.0,37.5,0.0,0.5,5.0,2.5,63.0,29.0,2.5,0.0


## step2) 사용자 특성 매트릭스(Weighted genre matrix)로 부터 User Profile 작성

가중 장르 행렬로부터 사용자 프로파일 작성 - 각 사용자의 장르 선호도를 나타내는 사용자 프로파일 작성

In [21]:
# 사용자 프로파일
# 각 사용자의 특성 벡터를 해당 벡터의 합으로 나누어 정규화합니다.
# 즉, 각 사용자가 선호하는 장르의 가중치를 전체 가중치 합으로 나누어 비율로 나타냅니다.
users_features = users_features / np.sum(users_features, axis=1, keepdims=True)

# 정규화된 사용자 특성 행렬 
users_features

array([[0.        , 0.12838284, 0.12310231, ..., 0.07524752, 0.03267327,
        0.00990099],
       [0.        , 0.1489726 , 0.04280822, ..., 0.12671233, 0.01541096,
        0.0119863 ],
       [0.        , 0.19230769, 0.11538462, ..., 0.11153846, 0.00961538,
        0.        ],
       ...,
       [0.        , 0.12643048, 0.07990132, ..., 0.12553964, 0.00931954,
        0.00397451],
       [0.        , 0.11447811, 0.10774411, ..., 0.15488215, 0.04713805,
        0.01346801],
       [0.        , 0.13587591, 0.07222628, ..., 0.1330292 , 0.0129562 ,
        0.0090146 ]])

## step3) 사용자가 보지 않은 영화에 사용자 프로파일을 적용하여 추천 영화 list 작성
- 사용자가 보지 않은 영화의 장르 행렬을 작성합니다.  
- 사용자 프로파일과 영화의 장르 행렬을 기반으로 각 영화에 대한 예측 평점을 계산합니다.

각 사용자별로 선호 장르를 순서대로 나열

In [22]:
# 특징 벡터의 길이 계산 - 사용자가 선호하는 장르의 수 의미
num_features = len(features)

# 각 사용자에 대해 상위 k개의 특징 인덱스를 추출
# tf.nn.top_k 함수는 주어진 배열에서 가장 큰 값의 인덱스를 반환
# users_features: 사용자의 장르 선호도를 나타내는 정규화된 특성 벡터
# k=num_features: 각 사용자의 모든 장르에 대한 인덱스를 반환합니다.
top_users_features = tf.nn.top_k(users_features, k=num_features)[1]

# 텐서플로우 텐서를 넘파이 배열로 변환하여 출력
top_users_features.numpy()

array([[ 1,  2,  5, ...,  0,  7, 12],
       [ 8,  1,  6, ...,  9, 10, 13],
       [16,  1, 11, ..., 10, 12, 19],
       ...,
       [ 5,  8,  1, ...,  7, 10,  0],
       [ 8, 17,  1, ..., 10, 13, 14],
       [ 1,  8, 17, ..., 10,  7,  0]])

장르 index 를 장르명으로 표시

In [23]:
# 상위 5명의 사용자를 처리
for i in range(5):  
    # 각 사용자의 상위 특징 인덱스를 사용하여 해당 특징(장르) 이름을 가져옴
    feature_names = [features[idx] for idx in top_users_features[i]]
    
    # 사용자 번호와 해당 사용자가 선호하는 상위 장르 목록을 출력
    print('사용자 {}: {}'.format(i, feature_names))
    print()  # 한 줄 띄워서 출력

사용자 0: ['Action', 'Adventure', 'Comedy', 'Drama', 'Thriller', 'Fantasy', 'Crime', 'Children', 'Sci-Fi', 'Animation', 'Romance', 'Musical', 'War', 'Mystery', 'Horror', 'Western', 'Film-Noir', '(no genres listed)', 'Documentary', 'IMAX']

사용자 1: ['Drama', 'Action', 'Crime', 'Thriller', 'Comedy', 'Sci-Fi', 'IMAX', 'Documentary', 'Adventure', 'Mystery', 'Romance', 'War', 'Western', 'Horror', '(no genres listed)', 'Animation', 'Children', 'Fantasy', 'Film-Noir', 'Musical']

사용자 2: ['Sci-Fi', 'Action', 'Horror', 'Adventure', 'Thriller', 'Fantasy', 'Drama', 'Comedy', 'Mystery', 'Children', 'Romance', 'War', 'Animation', 'Crime', 'Musical', '(no genres listed)', 'Documentary', 'Film-Noir', 'IMAX', 'Western']

사용자 3: ['Drama', 'Comedy', 'Romance', 'Thriller', 'Adventure', 'Crime', 'Action', 'Mystery', 'Fantasy', 'Musical', 'Children', 'Western', 'Sci-Fi', 'War', 'Animation', 'Horror', 'Film-Noir', 'Documentary', 'IMAX', '(no genres listed)']

사용자 4: ['Drama', 'Comedy', 'Crime', 'Children', 'Rom

### 추천할 영화 결정

- 이제 위에서 계산 한 `users_features` 를 사용하여 각 사용자에 대한 영화 등급 및 추천을 결정합니다.  
- 각 영화의 예상 등급을 계산하기 위해 사용자의 특성 벡터와 해당 영화 특성 벡터간의 유사성 측정값을 계산합니다. 

### 모든 영화 특성과 모든 사용자 특성간의 유사도 측정

In [24]:
# 모든 사용자의 예상 평점을 코사인 유사도로 계산
# users_features: 사용자의 장르 선호도를 나타내는 정규화된 특성 벡터
# movies_features: 각 영화의 장르 특성을 나타내는 벡터
# cosine_similarity: 두 벡터 간의 코사인 유사도를 계산하여 사용자가 각 영화를 얼마나 선호할지를 예측
all_users_ratings = cosine_similarity(users_features, movies_features)

print(all_users_ratings.shape)

(610, 9724)


In [25]:
# 모든 사용자의 예상 평점을 데이터프레임으로 변환
# columns에는 영화 이름 목록을 지정하여 각 열이 특정 영화를 나타내도록 함
df_all_users_ratings = pd.DataFrame(all_users_ratings, columns=movie_names_list)

df_all_users_ratings.head(3)

,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
0,0.641015,0.504297,0.376548,0.510222,0.404806,0.535239,0.376548,0.454760,0.443576,0.651768,...,0.598086,0.358003,0.534585,0.155080,0.000000,0.616901,0.456237,0.351212,0.423314,0.404806
1,0.174596,0.069569,0.221530,0.548200,0.269912,0.659510,0.221530,0.085204,0.419327,0.517590,...,0.419327,0.449876,0.640732,0.000000,0.125316,0.344619,0.155834,0.636220,0.296509,0.269912
2,0.253993,0.264624,0.081024,0.135188,0.089676,0.460215,0.081024,0.228981,0.498197,0.627043,...,0.617765,0.098638,0.147957,0.019928,0.000000,0.371157,0.140941,0.119567,0.366370,0.089676


In [37]:
# 모든 사용자의 예상 평점 DF의 열 이름을 영화 제목 리스트로 변환
all_users_ratings_titles = df_all_users_ratings.columns.tolist()

# 영화 제목 리스트의 길이(영화의 총 수) 출력
print(len(all_users_ratings_titles))
print(all_users_ratings_titles[:5])

9724
['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)', 'Waiting to Exhale (1995)', 'Father of the Bride Part II (1995)']


- 위의 계산은 데이터베이스의 각 사용자와 각 영화 간의 유사성 측정값을 계산합니다.  
- 새 영화의 등급에만 집중하기 위해 all_users_ratings 매트릭스에 마스크를 적용하여, 사용자가 이미 영화를 평가한 경우 해당 평가를 무시합니다. 이렇게 하면 이전에 본 적이 없거나 등급이 지정되지 않은 영화에 대한 등급에만 집중합니다.

In [32]:
# 사용자가 보지 않은 영화를 찾기 위해 이진 행렬을 생성
# users_seen_movies와 같은 형태의 0으로 된 배열을 생성하여 사용자가 보지 않은 영화를 찾습니다.
# np.equal 함수는 두 배열의 요소를 비교하여 같은 위치에 같은 값이 있는 경우 True를 반환합니다.
users_unseen_movies = np.equal(users_seen_movies, np.zeros_like(users_seen_movies))

# 사용자가 보지 않은 영화에 대한 이진 행렬 
users_unseen_movies

array([[False,  True, False, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True]])

In [33]:
users_ratings_expected = np.where(
    users_unseen_movies,   # 조건: 사용자가 영화를 보지 않았는지 여부 (True: 안 본 영화, False: 본 영화)
    all_users_ratings,     # True일 때 적용할 값: 모든 사용자의 평점(사용자가 보지 않은 영화에 대한 평점)
    0)         # False일 때 적용할 값: 무시할 값(이미 본 영화에 대해서는 변경하지 않음)
users_ratings_expected

array([[0.        , 0.50429694, 0.        , ..., 0.35121165, 0.42331387,
        0.40480563],
       [0.17459573, 0.06956855, 0.2215297 , ..., 0.63622031, 0.29650898,
        0.26991165],
       [0.25399269, 0.2646236 , 0.0810241 , ..., 0.11956735, 0.36636983,
        0.08967551],
       ...,
       [0.        , 0.        , 0.        , ..., 0.43293558, 0.34795643,
        0.43698381],
       [0.        , 0.23434694, 0.27301447, ..., 0.63360126, 0.25901373,
        0.22770045],
       [0.        , 0.23927991, 0.3337287 , ..., 0.44296564, 0.35760866,
        0.36599559]])

- 각 사용자에 대해 보지 않은 영화의 상위 2 개 등급의 영화를 가져와 인쇄해 봅니다.

In [34]:
# 각 사용자별로 예상 평점이 가장 높은 상위 2개의 영화 인덱스를 추출
# tf.nn.top_k 함수는 주어진 배열에서 가장 큰 값의 상위 k개의 인덱스를 반환합니다.
# users_ratings_expected: 사용자의 예상 평점 행렬
# k=2: 상위 2개의 예상 평점을 가진 영화의 인덱스를 반환하도록 설정합니다.
top_movies = tf.nn.top_k(users_ratings_expected, 2)[1]

# 상위 2개의 영화 인덱스 출력
top_movies

<tf.Tensor: shape=(610, 2), dtype=int32, numpy=
array([[8579, 6553],
       [  19,  118],
       [6772,  901],
       ...,
       [6553,   19],
       [6553, 4170],
       [6412,   19]])>

In [35]:
#df_all_users_ratings

In [36]:
# 추천할 영화 목록 생성 및 출력
for i in range(5):  # 상위 5명의 사용자에 대해서만 처리
    # 각 사용자의 상위 영화 인덱스를 사용하여 해당 영화 제목을 가져옴
    movie_names = [all_users_ratings_titles[idx] for idx in top_movies[i]]
    
    # 사용자 번호와 해당 사용자에게 추천할 상위 영화 목록을 출력
    print('사용자 {}: {}'.format(i, movie_names))

사용자 0: ['Dragonheart 2: A New Beginning (2000)', 'Hunting Party, The (2007)']
사용자 1: ['Money Train (1995)', 'Bad Boys (1995)']
사용자 2: ['Rogue (2007)', 'Aliens (1986)']
사용자 3: ['Waiting to Exhale (1995)', 'American President, The (1995)']
사용자 4: ['Osmosis Jones (2001)', 'Out of Sight (1998)']
